In [1]:
#Dataset is from
#https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [4]:
!unzip \*.zip && rm *.zip

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [5]:
!ls

2023-01-07_2015_submission.csv	ndtv_data_final.csv    test_features.csv
Chronic_Kidney_Disease.rar	o2Saturation.csv       train_features.csv
dataset.gsheet			onehr.data	       train_labels.csv
heart.csv			README.docx	       water_potability.csv
__MACOSX			sample_submission.csv
model_rf_rs_80			submission.csv


In [6]:
def wrangle(filepath):
  heart = pd.read_csv('heart.csv')

  heart.rename(columns = {'cp':'chest_pain_type', 'trtbps':'resting_heart_beat','chol':'cholestoral','fbs':'fasting_blood_sugar_>_120',
  'restecg':'resting_electrocardiographic',
  'thalachh':'maximum_heart_rate_achieved', 'exng':'exercise_induced_angina',
  'oldpeak':'Previous_Peak','slp':'slope','caa':'number_of_major_vessels','thall':'Thal_rate'}, inplace=True)

  return heart


In [7]:
ox = pd.read_csv('o2Saturation.csv')

In [8]:
heart = wrangle('heart.csv')

In [9]:
heart

,age,sex,chest_pain_type,resting_heart_beat,cholestoral,fasting_blood_sugar_>_120,resting_electrocardiographic,maximum_heart_rate_achieved,exercise_induced_angina,Previous_Peak,slope,number_of_major_vessels,Thal_rate,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [10]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           303 non-null    int64  
 1   sex                           303 non-null    int64  
 2   chest_pain_type               303 non-null    int64  
 3   resting_heart_beat            303 non-null    int64  
 4   cholestoral                   303 non-null    int64  
 5   fasting_blood_sugar_>_120     303 non-null    int64  
 6   resting_electrocardiographic  303 non-null    int64  
 7   maximum_heart_rate_achieved   303 non-null    int64  
 8   exercise_induced_angina       303 non-null    int64  
 9   Previous_Peak                 303 non-null    float64
 10  slope                         303 non-null    int64  
 11  number_of_major_vessels       303 non-null    int64  
 12  Thal_rate                     303 non-null    int64  
 13  outpu

Define Target

In [11]:
target = 'output'
X = heart.drop(columns = target)
y =heart[target]

In [12]:
y_pred = [y.mean()] *len(y)
mean_absolute_error(y,y_pred)

0.49602980099990196

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

model_rf = RandomForestClassifier(random_state=0)

model_rf.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [ ]:
import seaborn as sns
sns.pairplot(data=heart)

# Model Score

In [16]:
y_pred = model_rf.predict(X_test)

mean_absolute_error(y_test,y_pred)

0.14754098360655737

In [17]:
model_rf.score(X_test,y_test)

0.8524590163934426

In [18]:
import plotly.express as px
fig = px.scatter(x=heart['age'], y=heart['maximum_heart_rate_achieved'], trendline='ols')
fig.show()

Tune model

In [41]:
clf = make_pipeline(
    RandomForestClassifier(random_state=42,n_jobs=-1,max_depth=5,n_estimators=100)
)

param_grid = {'randomforestclassifier__max_depth':range(5,40,5),
              'randomforestclassifier__n_estimators':range(25,125,25)}

model = RandomizedSearchCV(clf, param_distributions=param_grid, n_jobs=-1,
                           verbose=1, n_iter=25,cv=5)

model.fit(X,y)

model.best_estimator_

Fitting 5 folds for each of 25 candidates, totalling 125 fits


Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_depth=30, n_estimators=75,
                                        n_jobs=-1, random_state=42))])

In [42]:
model_rf2 = RandomForestClassifier(max_depth = 5, n_estimators=75, n_jobs=1, random_state=42)

model_rf2.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=75, n_jobs=1, random_state=42)

In [43]:
pred2 = model_rf2.predict(X_test)

In [46]:
print("Model mean absolute error",mean_absolute_error(y_test, pred2))

Model mean absolute error 0.11475409836065574


In [47]:
print("Model score",model_rf2.score(X_test,y_test))

Model score 0.8852459016393442
